In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility関数


In [2]:
def save_metrics(metrics, epoch=None):
    plt.figure(figsize=(10,8))
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join("metrics", "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join("metrics", "g_loss" + str(epoch) + ".png"))
    plt.close()

In [3]:
# noise[[examples, 100]]から生成した画像をplot_dim(例えば4x4)で表示
def save_imgs(images, plot_dim=(5,12), size=(12,5), epoch=None):
    examples = plot_dim[0]*plot_dim[1]

    # 表示
    fig = plt.figure(figsize=size)
    for i in range(examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((96, 96, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join("generated_figures", str(epoch) + ".png"))
    plt.close()


# モデル

In [4]:
class Generator:
    def __init__(self, depths=[512,256,128,64], s_size=6):
        self.depths = depths + [3]
        self.s_size = s_size
        self.reuse = False

    def __call__(self, inputs, training=False):
        def leaky_relu(x, leak=0.3, name=''):
            return tf.maximum(x, x * leak, name=name)

        inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            with tf.variable_scope('reshape'):
                outputs = tf.layers.dense(inputs, self.depths[0] * self.s_size * self.s_size)
                outputs = tf.reshape(outputs, [-1, self.s_size, self.s_size, self.depths[0]])
                #outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            # deconvolution (transpose of convolution) x 4
            with tf.variable_scope('deconv1'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                #outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv2'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                #outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv3'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
#                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv4'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
            # output images
            with tf.variable_scope('tanh'):
                outputs = tf.tanh(outputs, name='outputs')
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [5]:
class Discriminator:
    def __init__(self, depths=[64,128,256,512]):
        self.depths = [3] + depths
        self.reuse = False

    def __call__(self, inputs, training=False, name=''):
        def leaky_relu(x, leak=0.3, name=''):
            return tf.maximum(x, x * leak, name=name)
        outputs = tf.convert_to_tensor(inputs)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            # convolution x 4
            with tf.variable_scope('conv1'):
                outputs = tf.layers.conv2d(outputs, self.depths[1], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv2'):
                outputs = tf.layers.conv2d(outputs, self.depths[2], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv3'):
                outputs = tf.layers.conv2d(outputs, self.depths[3], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv4'):
                outputs = tf.layers.conv2d(outputs, self.depths[4], [5, 5], strides=(2, 2), padding='SAME')
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('classify'):
                batch_size = outputs.get_shape()[0].value
                reshape = tf.reshape(outputs, [batch_size, -1])
                outputs = tf.layers.dense(reshape, 2, name='outputs')
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs


In [6]:
class DCGAN:
    def __init__(self,
                 batch_size=64, s_size=6, z_dim=100,
                 g_depths=[1024, 512, 256, 128],
                 d_depths=[64, 128, 256, 512]):
        self.batch_size = batch_size
        self.s_size = s_size
        self.z_dim = z_dim
        self.g = Generator(depths=g_depths, s_size=self.s_size)
        self.d = Discriminator(depths=d_depths)
        self.z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0)

    def loss(self, traindata):
        """build models, calculate losses.
        Args:
            traindata: 4-D Tensor of shape `[batch, height, width, channels]`.
        Returns:
            dict of each models' losses.
        """
        generated = self.g(self.z, training=True)
        g_outputs = self.d(generated, training=True, name='g')
        t_outputs = self.d(traindata, training=True, name='t')
        # add each losses to collection
        tf.add_to_collection(
            'g_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.ones([self.batch_size], dtype=tf.int64),
                    logits=g_outputs)))
        tf.add_to_collection(
            'd_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.ones([self.batch_size], dtype=tf.int64),
                    logits=t_outputs)))
        tf.add_to_collection(
            'd_losses',
            tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=tf.zeros([self.batch_size], dtype=tf.int64),
                    logits=g_outputs)))
        return {
            self.g: tf.add_n(tf.get_collection('g_losses'), name='total_g_loss'),
            self.d: tf.add_n(tf.get_collection('d_losses'), name='total_d_loss'),
        }


    def train(self, losses, learning_rate=1e-4, beta1=0.5):
        g_optimizer = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=beta1)
        d_optimizer = tf.train.AdamOptimizer(learning_rate=2e-4, beta1=beta1)

        g_optimizer = g_optimizer.minimize(losses[self.g], var_list=self.g.variables)
        d_optimizer = d_optimizer.minimize(losses[self.d], var_list=self.d.variables)
        with tf.control_dependencies([g_optimizer, d_optimizer]):
            return tf.no_op(name="train")

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        #save_imgs(images, epoch=epoch)
        return images



In [ ]:
import numpy as np

X_train = np.load("irasutoya_10575x96x96x3_jpg.npy")
X_train = X_train/255

z_dim = 100
batch_size = 64
epochs = 100000
steps = 5
display_epoch = 250
param_save_epoch = 10000
loss = {"dis_loss":[], "gen_loss":[]}

In [ ]:
p_noise = tf.placeholder(tf.float32, [None, z_dim])
noise_check = np.random.uniform(-1, 1, size=[60, z_dim]).astype(np.float32)

dcgan = DCGAN()
p_train_images = tf.placeholder(tf.float32, [batch_size, 96, 96, 3])
losses = dcgan.loss(traindata=p_train_images)
train_op = dcgan.train(losses)

saver = tf.train.Saver()
#cache = dcgan.cache_d_weights()
#load = dcgan.load_d_weights()


#assign = tf.assign(d, s)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        
        if epoch % display_epoch == 0:
            imgs_gen = dcgan.sample_images(inputs=noise_check).eval()
            print("saving images")
            save_imgs(imgs_gen, epoch=epoch)

        dis_var = []
        for step in range(steps):
            # ノイズから生成データを生成
            noise = np.random.uniform(-1, 1, size=[batch_size, z_dim])

            # 訓練データを抜粋
            rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)
            exImgs = X_train[rand_index, :].astype(np.float32)

            _, g_loss_value, d_loss_value = sess.run([train_op, losses[dcgan.g], losses[dcgan.d]], feed_dict={p_train_images: exImgs})

            if step == 0:
                # 初回だけdiscriminatorのvariableを保存
                dis_var = sess.run(dcgan.d.variables)

        np_dis_var = np.array(dis_var)
        for (dst, src) in zip(dcgan.d.variables, np_dis_var):
            #sess.run(assign, feed_dict={d:dst, s:src})
            #sess.run(tf.assign(dst, src))
            dst = src
        #np_dis_var = None
        sess.run(dcgan.d.variables)
        del np_dis_var

        # 結果をappend
        loss["dis_loss"].append(d_loss_value)
        loss["gen_loss"].append(g_loss_value)
        print("epoch:" + str(epoch))
        # グラフの描画（余裕があったら）
        if epoch % display_epoch == 0:
            save_metrics(loss, epoch)

        if epoch % param_save_epoch == 0:
            saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")

saving images
epoch:0
epoch:1
epoch:2
epoch:3
epoch:4
epoch:5
epoch:6
epoch:7
epoch:8
epoch:9
epoch:10
epoch:11
epoch:12
epoch:13
epoch:14
epoch:15
epoch:16
epoch:17
epoch:18
epoch:19
epoch:20
epoch:21
epoch:22
epoch:23
epoch:24
epoch:25
epoch:26
epoch:27
epoch:28
epoch:29
epoch:30
epoch:31
epoch:32
epoch:33
epoch:34
epoch:35
epoch:36
epoch:37
epoch:38
epoch:39
epoch:40
epoch:41
epoch:42
epoch:43
epoch:44
epoch:45
epoch:46
epoch:47
epoch:48
epoch:49
epoch:50
epoch:51
epoch:52
epoch:53
epoch:54
epoch:55
epoch:56
epoch:57
epoch:58
epoch:59
epoch:60
epoch:61
epoch:62
epoch:63
epoch:64
epoch:65
epoch:66
epoch:67
epoch:68
epoch:69
epoch:70
epoch:71
epoch:72
epoch:73
epoch:74
epoch:75
epoch:76
epoch:77
epoch:78
epoch:79
epoch:80
epoch:81
epoch:82
epoch:83
epoch:84
epoch:85
epoch:86
epoch:87
epoch:88
epoch:89
epoch:90
epoch:91
epoch:92
epoch:93
epoch:94
epoch:95
epoch:96
epoch:97
epoch:98
epoch:99
epoch:100
epoch:101
epoch:102
epoch:103
epoch:104
epoch:105
epoch:106
epoch:107
epoch:108
epoch: